# Ballot Polling Assertion RLA Examples

In [1]:
from __future__ import division, print_function

import math
import json
import warnings
import numpy as np
import copy

from collections import OrderedDict

from assertion_audit_utils import \
    Assertion, Assorter, CVR, TestNonnegMean, check_audit_parameters,\
    find_p_values, summarize_status

## Example 1: small election, large margin

This election is between Alice and Bob and contains 100 ballots. The reported winner is Alice with 70% of the valid votes. 

Candidate | total 
---|---
Alice | 70
Bob | 30
Ballots | 100


A sample of 30 ballots (simulated from ASN) is drawn from 100 ballots with and without replacement. 73.33% (22/30 cards) of the sample is for Alice and 26.67% (8/30 cards) of the ballots is for Bob. The example audits are done with and without replacement using the `wald_sprt` and `kaplan_kolmogorov` risk functions respectively. 

In [2]:
contests = {'339':{'risk_limit': 0.05,
                        'choice_function': 'plurality',
                        'n_winners': 1,
                        'candidates': ['Alice', 'Bob'],
                        'reported_winners': ['Alice']
                        }
            }

mvr_file = './Data/mvr_polling_example_1.json'
manifest_type = 'STYLE'

In [3]:
all_assertions = Assertion.make_all_assertions(contests)

In [4]:
with open(mvr_file) as f:
    mvr_json = json.load(f)

mvr_sample = CVR.from_dict(mvr_json['ballots'])

### With replacement


In [5]:
risk_fn = lambda x: TestNonnegMean.wald_sprt(np.array(x), N=N_cards, t=t, p1=p1, random_order=False)

N_cards = np.inf
t=1/2
p1=0.7

In [6]:
p_max = find_p_values(contests, all_assertions, risk_fn, manifest_type, mvr_sample)
print("maximum assertion p-value {}".format(p_max))
np.testing.assert_almost_equal(p_max, (t/p1)**22 * np.divide(1-t, 1-p1)**8)

maximum assertion p-value 0.036305566021550766


### Without replacement

In [7]:
risk_fn = lambda x: TestNonnegMean.kaplan_kolmogorov(np.array(x), N=N_cards, t=t, g=g, random_order=False)

N_cards = 100
t=1/2
g=0.1

In [8]:
p_max = find_p_values(contests, all_assertions, risk_fn, manifest_type, mvr_sample)
print("maximum assertion p-value {}".format(p_max))

np.testing.assert_almost_equal(p_max, 1.1**-22 * 0.6 * np.prod(np.divide(np.linspace(60, 38, 21), \
                                np.arange(79, 100))) * 0.1**-8 * (np.prod(np.divide(np.linspace(36.9, 36.2, 8), \
                                    np.arange(71, 79)))))


maximum assertion p-value 0.07715494796551615


## Example 2: Medium election, large margin

This election is between Alice and Bob and contains 10000 ballots. The reported winner is Alice with 60% of the valid votes. 

Candidate | total 
---|---
Alice | 6000
Bob | 4000
Ballots | 10000

A sample of 200 ballots is drawn from 10000 ballots with and without replacement. The sample shows 65% (130/200 cards) of the ballots are for Alice and 35% (70/200 cards) of the ballots are for Bob. The example audits are done with and without replacement using the `wald_sprt` and `kaplan_kolmogorov` risk functions respectively. 

In [9]:
#contests to audit
contests = {'339':{'risk_limit': 0.05,
                        'choice_function': 'plurality',
                        'n_winners': 1,
                        'candidates': ['Alice', 'Bob'],
                        'reported_winners': ['Alice']
                        }
            }

mvr_file = './Data/mvr_polling_example_2.json'
manifest_type = 'STYLE'

In [10]:
all_assertions = Assertion.make_all_assertions(contests)

In [11]:
with open(mvr_file) as f:
    mvr_json = json.load(f)

mvr_sample = CVR.from_dict(mvr_json['ballots'])

### With replacement

In [12]:
risk_fn = lambda x: TestNonnegMean.wald_sprt(np.array(x), N=N_cards, t=t, p1=p1, random_order=False)

N_cards = np.inf
t=1/2
p1=0.60

In [13]:
p_max = find_p_values(contests, all_assertions, risk_fn, manifest_type, mvr_sample)
print("maximum assertion p-value {}".format(p_max))
np.testing.assert_almost_equal(p_max, (t/p1)**130 * np.divide(1-t, 1-p1)**70)

maximum assertion p-value 0.0003091284130380668


### Without replacement

In [14]:
risk_fn = lambda x: TestNonnegMean.kaplan_kolmogorov(np.array(x), N=N_cards, t=t, g=g, random_order=False)

N_cards = 10000
t=1/2
g=0.5

In [15]:
p_max = find_p_values(contests, all_assertions, risk_fn, manifest_type, mvr_sample)
print("maximum assertion p-value {}".format(p_max))

np.testing.assert_almost_equal(p_max, 1.5**-130 * np.prod(np.divide(np.linspace(10000, 9808, 129), \
                                np.arange(9871, 10000))) * 0.5**-70 * (np.prod(np.divide(np.linspace(9806.5, \
                                    9772, 70), np.arange(9801, 9871)))))


maximum assertion p-value 0.00726793364131293
